In [1]:
# pdf를 위한 Loader, docx를 위한 loader... 등 각각의 loader를 찾는 대신 UnstructuredFileLoader를 사용할 수 있음, 각각의 파일을 열기 위해 패키지들을 다운로드할수도 있음
from langchain.document_loaders import  TextLoader, PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

#CharaterTextSplitter는 seperator가 따로 있어서 특정 문자열을 찾은 다음에 거기부터 끊는다.
#문장의 끝이나 문단의 끝부분마다 끊어준다. 최대한 문장 중간에서 분할하지 않는다.
splitter = RecursiveCharacterTextSplitter(
  chunk_size=200,
  #위에것만 사용하면 문단 중간에 그냥 잘릴수도 있음, 그래서 아래를 사용해 문장이나 문단을 분할할 때 앞 조각 일부분을 가져옥게 만든다. 문서 사이에 겹치는 부분이 생기게됨(어떤 문서의 끝부분이 다른 문서의 시작점이 된다.)
  chunk_overlap=50
)

splitter2 = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  #글자 갯수 600 기준 seperator를 기준으로 자름
  chunk_size=600,
  chunk_overlap=100,
  length_function=len
)


loader = TextLoader("./files/chapter_one.txt")

"""
- data connection
    - source
    - transform- split
    - embedded
    - store
    - retrieve
"""
#문서가 너무 크기 때문에 나누는 작업을 한다.
#파일을 전달할 때 질문에 답해야할 때 필요한 파일의 부분들만을 전달할 수 있다.
#loader.load()

docs = loader.load()

loader.load_and_split(text_splitter=splitter)

In [ ]:

#token!=letter

#embedding- 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업, 벡터를 만드는 작업
#https://www.youtube.com/watch?v=2eWuYf-aZE4 영상 참고
#단어를 숫자로 치환하면 단어끼리 연산을 할 수 있게됨
#문서를 숫자로 바꾸면 벡터에대한 검색을 할 수 있게 된다. 추천 알고리즘이 동작하는 방식
# 어떤 벡터가 가까운지 알 수 있고


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

#hi를 표현하는 vector
vector = embedder.embed_query("Hi")
embedder.embed_documents(["hii", "how", "are", "you"])

In [ ]:
#매번 코드를 실행할때마다 embed를 실행하는것은 좋은것이 아니기 때문에 Embedding을 저장한다.
#vector store는 일종의 데이터베이스로 벡터 공간에서 검색할 수 있게 해준다.
#벡터를 생성하고 캐시해서 스토어에 넣어주면 검색할 수 있다.관련있는 문서만을 찾을 수 있음
#chroma

from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore


cache_dir = LocalFileStore("./.cache/")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

#캐시를 먼저 확인
vectorstore = Chroma.from_documents(docs, cached_embeddings)

#문서는 모두 벡터로 바뀌었고 이제 벡터 공간에서 검색할 수 있다.
vectorstore.similarity_search("where does winston live")

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  #document를 ㅁ낳은 장소로부터 선별하여 가져옴
  retriever=vectorstore.as_retriever()
)

chain.run("where does winston live?")